In [167]:
import pandas as pd
import numpy as np

In [168]:
#data = pd.read_csv("../../merged_dataset_ellie.csv")
data = pd.read_csv("../../data/combined_jun22_train.csv", encoding = "ISO-8859-1")

In [169]:
pd.set_option('max_columns', None)

In [170]:
data.sample(10)

,Unnamed: 0,ID,YEAR_FIPS,YEAR,STATE_FIPS,COUNTY_FIPS,STATE,COUNTY,REP_CANDIDATE,DEM_CANDIDATE,REP_VOTES,DEM_VOTES,COUNTY_TOTALVOTES,WINNING_CANDIDATE,WINNING_PARTY,WINNING_PARTY_BINARY,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,MARGIN_VICTORY,TRAIN
9751,9751,12980,200417131,2004,17,131,Illinois,Mercer County,George W. Bush,John Kerry,4405.0,4512.0,8974.0,John Kerry,democrat,0,0,0,6.5,195832.0,24,10,23,39,124,115,10,11,1,0,8568,8261,16829,8461,8158,4517,2283,2234,3739,1837,1902,4945,2514,2431,3628,1627,2001,0.001426,0.000594,0.001367,0.002317,0.007368,0.006833,0.000594,0.000654,0.000059,0.000000,0.502763,0.484758,0.509121,0.490879,0.268406,0.222176,0.293838,0.215580,0.011923,0
8030,8030,11255,200039013,2000,39,13,Ohio,Belmont County,George W. Bush,Al Gore,12625.0,15980.0,30141.0,Al Gore,democrat,0,1,1,6.4,22159.0,115,105,693,1895,126,148,47,52,9,4,35756,34483,70239,34662,32166,16927,8784,8143,17177,9211,7966,19992,10028,9964,16143,6460,9683,0.001637,0.001495,0.009866,0.026979,0.001794,0.002107,0.000669,0.000740,0.000128,0.000057,0.493487,0.457951,0.509062,0.490938,0.240991,0.244551,0.284628,0.229830,0.111310,0
10113,10113,13349,200421053,2004,21,53,Kentucky,Clinton County,George W. Bush,John Kerry,3369.0,952.0,4352.0,George W. Bush,republican,1,1,1,6.8,0.0,4,1,5,13,59,104,8,17,5,13,5048,4777,9825,5007,4705,2454,1231,1223,2453,1232,1221,2798,1390,1408,2120,924,1196,0.000407,0.000102,0.000509,0.001323,0.006005,0.010585,0.000814,0.001730,0.000509,0.001323,0.509618,0.478880,0.513791,0.486209,0.249771,0.249669,0.284784,0.215776,-0.555377,0
11399,11399,14654,200442093,2004,42,93,Pennsylvania,Montour County,George W. Bush,John Kerry,4903.0,2666.0,7624.0,George W. Bush,republican,1,0,1,5.2,39820.0,126,129,93,130,109,112,4,9,1,0,9518,8721,18239,9256,8398,4649,2403,2246,3964,1930,2034,5501,2684,2817,4125,1704,2421,0.006908,0.007073,0.005099,0.007128,0.005976,0.006141,0.000219,0.000493,0.000055,0.000000,0.507484,0.460442,0.521849,0.478151,0.254893,0.217336,0.301606,0.226164,-0.293416,0
3049,3049,3419,201249021,2012,49,21,Utah,Iron County,Mitt Romney,Barack Obama,14200.0,2148.0,16701.0,Mitt Romney,republican,1,1,1,6.8,0.0,228,184,107,155,1747,1840,574,532,74,87,23265,22998,46263,21843,21613,16289,8126,8163,14517,7310,7207,8998,4419,4579,6459,3143,3316,0.004928,0.003977,0.002313,0.003350,0.037762,0.039773,0.012407,0.011499,0.001600,0.001881,0.472148,0.467177,0.502886,0.497114,0.352096,0.313793,0.194497,0.139615,-0.721633,0
8255,8255,11480,200042031,2000,42,31,Pennsylvania,Clarion County,George W. Bush,Al Gore,9796.0,5605.0,15843.0,George W. Bush,republican,1,1,1,5.0,23956.0,68,78,154,180,99,73,24,21,1,1,21591,20148,41739,21252,19773,11228,5553,5675,11741,5758,5983,10490,5210,5280,8280,3627,4653,0.001629,0.001869,0.003690,0.004313,0.002372,0.001749,0.000575,0.000503,0.000024,0.000024,0.509164,0.473730,0.517286,0.482714,0.269005,0.281296,0.251324,0.198376,-0.264533,0
15209,15209,18494,20085145,2008,5,145,Arkansas,White County,John McCain,Barack Obama,19467.0,6732.0,26955.0,John McCain,republican,1,1,0,6.0,165850.0,215,187,1430,1429,1031,1344,207,183,22,17,37602,35971,73573,35140,33621,20663,10455,10208,19827,9885,9942,19181,9497,9684,13902,6134,7768,0.002922,0.002542,0.019436,0.019423,0.014013,0.018268,0.002814,0.002487,0.000299,0.000231,0.477621,0.456975,0.511084,0.488916,0.280850,0.269487,0.260707,0.188955,-0.472454,0
6986,6986,10191,200021025,2000,21,25,Kentucky,Breathitt County

In [171]:
## some cleanup..

data.isnull().sum()

Unnamed: 0          0
ID                  0
YEAR_FIPS           0
YEAR                0
STATE_FIPS          0
                   ..
TOT_POP_20to39%     0
TOT_POP_40to59%     0
TOT_POP_Above60%    0
MARGIN_VICTORY      0
TRAIN               0
Length: 67, dtype: int64

In [172]:
data.isna().sum()

Unnamed: 0          0
ID                  0
YEAR_FIPS           0
YEAR                0
STATE_FIPS          0
                   ..
TOT_POP_20to39%     0
TOT_POP_40to59%     0
TOT_POP_Above60%    0
MARGIN_VICTORY      0
TRAIN               0
Length: 67, dtype: int64

In [173]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15569 entries, 0 to 15568
Data columns (total 67 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15569 non-null  int64  
 1   ID                     15569 non-null  int64  
 2   YEAR_FIPS              15569 non-null  int64  
 3   YEAR                   15569 non-null  int64  
 4   STATE_FIPS             15569 non-null  int64  
 5   COUNTY_FIPS            15569 non-null  int64  
 6   STATE                  15569 non-null  object 
 7   COUNTY                 15569 non-null  object 
 8   REP_CANDIDATE          15569 non-null  object 
 9   DEM_CANDIDATE          15569 non-null  object 
 10  REP_VOTES              15569 non-null  float64
 11  DEM_VOTES              15569 non-null  float64
 12  COUNTY_TOTALVOTES      15569 non-null  float64
 13  WINNING_CANDIDATE      15569 non-null  object 
 14  WINNING_PARTY          15569 non-null  object 
 15  WI

# cleaning/wrangling...

In [174]:
from sklearn.model_selection import train_test_split

In [176]:
def train_test_split_by_year_2(X, year, train_filter=False, cols=None):
    
    X = data.drop("WINNING_PARTY_BINARY", axis=1)
    y = data[["WINNING_PARTY_BINARY", "TRAIN"]]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
    
    if (year == None):
        #then we're done.
        return X_train, X_test, y_train, y_test
    
    filter_year =  X['YEAR'] == year
    not_filter_year =  X['YEAR'] != year
    
    X_test = X_test[filter_year]
    
    if (cols != None):
        X_test = X_test[cols]
    
    print('X_test: {}'.format(X_test.shape))
    
    y_test = y_test[filter_year]
    y_test = y_test.drop("TRAIN", axis=1)
    print('y_test: {}'.format(y_test.shape))

    X_train = X_train[not_filter_year]

    if (train_filter == True):
        X_train = X_train.query('TRAIN == 1')
        X_train = X_train.reset_index(drop=True)
        
    X_train.drop("TRAIN", axis=1)
    
    if (cols != None):
        X_train = X_train[cols]
   
    y_train = y_train[not_filter_year]
    
    if (train_filter == True):
        y_train = y_train.query('TRAIN == 1')
        y_train = y_train.reset_index(drop=True)
        
    y_train = y_train.drop("TRAIN", axis=1)
    
    print('X_train: {}'.format(X_train.shape))
    print('y_train: {}'.format(y_train.shape))

    return X_train, X_test, y_train, y_test

### Experiment 1 - Train with the data as is -- i.e without balancing training data. 

In [182]:
year = None #2016

In [183]:
data = pd.read_csv("../../data/combined_jun22_train.csv", encoding = "ISO-8859-1")

In [184]:
#add: REP_VOTES, DEM_VOTES, MARGIN_VICTORY
data.drop(['ID', 'Unnamed: 0', 'STATE_FIPS', 'COUNTY_FIPS', 'REP_VOTES', 'DEM_VOTES', 'MARGIN_VICTORY', 'WINNING_CANDIDATE', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'WINNING_PARTY', 'COUNTY', 'STATE'], axis=1, inplace=True)

In [185]:
X_train, X_test, y_train, y_test = train_test_split_by_year_2(data, year)


In [186]:
from sklearn.linear_model import LogisticRegression

In [187]:
model1 = LogisticRegression()

In [188]:
X_train.loc[X_train['YEAR'] == year]

,YEAR_FIPS,YEAR,COUNTY_TOTALVOTES,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,TRAIN


In [189]:
X_test.loc[X_test['YEAR'] == year]

,YEAR_FIPS,YEAR,COUNTY_TOTALVOTES,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,TRAIN


In [190]:
model1.fit(X_train, y_train)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: bad input shape (10898, 2)

In [145]:
X_train.columns

Index(['YEAR_FIPS', 'YEAR', 'COUNTY_TOTALVOTES', 'HOUSE_WINNING_BINARY',
       'SENATE_WINNING_BINARY', 'UNEMPLOYMENT_RATE', 'AVG_WAGE_SALARY',
       'AA_FEMALE', 'AA_MALE', 'BA_FEMALE', 'BA_MALE', 'H_FEMALE', 'H_MALE',
       'IA_FEMALE', 'IA_MALE', 'NA_FEMALE', 'NA_MALE', 'TOT_FEMALE',
       'TOT_MALE', 'TOT_POP', 'WA_FEMALE', 'WA_MALE', 'TOT_POP_LESS19',
       'TOT_MALE_LESS19', 'TOT_FEMALE_LESS19', 'TOT_POP_20to39',
       'TOT_MALE_20to39', 'TOT_FEMALE_20to39', 'TOT_POP_40to59',
       'TOT_MALE_40to59', 'TOT_FEMALE_40to59', 'TOT_POP_Above60',
       'TOT_MALE_Above60', 'TOT_FEMALE_Above60', 'AA_FEMALE%', 'AA_MALE%',
       'BA_FEMALE%', 'BA_MALE%', 'H_FEMALE%', 'H_MALE%', 'IA_FEMALE%',
       'IA_MALE%', 'NA_FEMALE%', 'NA_MALE%', 'WA_FEMALE%', 'WA_MALE%',
       'TOT_FEMALE%', 'TOT_MALE%', 'TOT_POP_LESS19%', 'TOT_POP_20to39%',
       'TOT_POP_40to59%', 'TOT_POP_Above60%', 'TRAIN'],
      dtype='object')

In [91]:
predictions = model1.predict(X_test)

In [92]:
from sklearn.metrics import accuracy_score

In [93]:
print("Experiment#1: {:.3f}%".format( accuracy_score(y_test, predictions) * 100))

Experiment#1: 87.252%


In [94]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.81      0.30      0.44       158
           1       0.88      0.99      0.93       799

   micro avg       0.87      0.87      0.87       957
   macro avg       0.84      0.64      0.68       957
weighted avg       0.87      0.87      0.85       957



In [95]:
from sklearn.metrics import confusion_matrix, classification_report

In [96]:
confusion_matrix(y_test, predictions)

array([[ 47, 111],
       [ 11, 788]])

### Experiment 2 - Balance training data 

In [98]:
data = pd.read_csv("../../data/combined_jun22_train.csv", encoding = "ISO-8859-1")

In [99]:
data.drop(['ID', 'Unnamed: 0', 'STATE_FIPS', 'COUNTY_FIPS', 'REP_VOTES', 'DEM_VOTES', 'MARGIN_VICTORY', 'WINNING_CANDIDATE', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'WINNING_PARTY', 'COUNTY', 'STATE'], axis=1, inplace=True)

In [100]:
### retest with Training=1 data.. 
X_train, X_test, y_train, y_test = train_test_split_by_year_2(data, year, True)

X_test: (957, 53)
y_test: (957, 1)
X_train: (433, 53)
y_train: (433, 1)


/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [101]:
model2 = LogisticRegression()

In [102]:
model2.fit(X_train, y_train)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [103]:
predictions = model2.predict(X_test)

In [104]:
print("Experiment#2: {:.3f}%".format( accuracy_score(y_test, predictions) * 100))

Experiment#2: 50.679%


In [105]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.24      0.92      0.38       158
           1       0.96      0.43      0.59       799

   micro avg       0.51      0.51      0.51       957
   macro avg       0.60      0.67      0.49       957
weighted avg       0.84      0.51      0.56       957



In [106]:
confusion_matrix(y_test, predictions)

array([[145,  13],
       [459, 340]])

### Experiment 3 - Expr2 + remove demographics absolute values

In [107]:
data = pd.read_csv("../../data/combined_jun22_train.csv", encoding = "ISO-8859-1")

In [108]:
data.drop(['ID', 'Unnamed: 0', 'STATE_FIPS', 'COUNTY_FIPS', 'WINNING_CANDIDATE', 'REP_VOTES', 'DEM_VOTES', 'MARGIN_VICTORY', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'WINNING_PARTY', 'COUNTY', 'STATE'], axis=1, inplace=True)

In [109]:
data.drop(['AA_FEMALE', 'AA_MALE', 'BA_FEMALE', 'BA_MALE', 'H_FEMALE', 'H_MALE', 'IA_FEMALE', 'IA_MALE', 'NA_FEMALE', 'NA_MALE' , 'TOT_FEMALE', 'TOT_MALE', 'TOT_POP', 'WA_FEMALE', 'WA_MALE', 'TOT_POP_LESS19', 'TOT_MALE_LESS19', 'TOT_FEMALE_LESS19', 'TOT_POP_20to39', 'TOT_MALE_20to39', 'TOT_FEMALE_20to39', 'TOT_POP_40to59', 'TOT_MALE_40to59', 'TOT_FEMALE_40to59', 'TOT_POP_Above60', 'TOT_MALE_Above60', 'TOT_FEMALE_Above60'] , axis=1, inplace=True)

In [110]:
X_train, X_test, y_train, y_test = train_test_split_by_year_2(data, year, True)

X_test: (957, 26)
y_test: (957, 1)
X_train: (433, 26)
y_train: (433, 1)


/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [111]:
model3 = LogisticRegression()

In [112]:
model3.fit(X_train, y_train)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [113]:
predictions = model3.predict(X_test)

In [114]:
print("Experiment#3: {:.3f}%".format( accuracy_score(y_test, predictions) * 100))

Experiment#3: 16.510%


In [115]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.17      1.00      0.28       158
           1       0.00      0.00      0.00       799

   micro avg       0.17      0.17      0.17       957
   macro avg       0.08      0.50      0.14       957
weighted avg       0.03      0.17      0.05       957



/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Experiment 4 - use significant columns only to build the model 


In [117]:
data = pd.read_csv("../../data/combined_jun22_train.csv", encoding = "ISO-8859-1")

In [118]:
data.drop(['ID', 'Unnamed: 0', 'STATE_FIPS', 'COUNTY_FIPS', 'REP_VOTES', 'DEM_VOTES', 'MARGIN_VICTORY', 'WINNING_CANDIDATE', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'WINNING_PARTY', 'COUNTY', 'STATE'], axis=1, inplace=True)
data.drop(['AA_FEMALE', 'AA_MALE', 'BA_FEMALE', 'BA_MALE', 'H_FEMALE', 'H_MALE', 'IA_FEMALE', 'IA_MALE', 'NA_FEMALE', 'NA_MALE' , 'TOT_FEMALE', 'TOT_MALE', 'TOT_POP', 'WA_FEMALE', 'WA_MALE', 'TOT_POP_LESS19', 'TOT_MALE_LESS19', 'TOT_FEMALE_LESS19', 'TOT_POP_20to39', 'TOT_MALE_20to39', 'TOT_FEMALE_20to39', 'TOT_POP_40to59', 'TOT_MALE_40to59', 'TOT_FEMALE_40to59', 'TOT_POP_Above60', 'TOT_MALE_Above60', 'TOT_FEMALE_Above60'] , axis=1, inplace=True)


In [119]:
X_train, X_test, y_train, y_test = train_test_split_by_year_2(data, year, True)

X_test: (957, 26)
y_test: (957, 1)
X_train: (433, 26)
y_train: (433, 1)


/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [120]:
X_train

,YEAR_FIPS,YEAR,COUNTY_TOTALVOTES,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,TRAIN
0,20125017,2012,4361.0,1,-1,11.4,30300.0,0.002627,0.002373,0.285254,0.260678,0.020763,0.025424,0.001356,0.001780,0.000254,0.000085,0.215085,0.224322,0.507881,0.492119,0.254407,0.227542,0.273390,0.244661,1
1,20045047,2004,7289.0,0,0,4.8,0.0,0.002568,0.002177,0.002345,0.005080,0.008597,0.010663,0.004968,0.004354,0.000391,0.000391,0.487467,0.479038,0.503433,0.496567,0.281137,0.239770,0.267515,0.211578,1
2,200848395,2008,6710.0,1,1,4.9,0.0,0.002418,0.002659,0.121721,0.105222,0.082497,0.083887,0.003808,0.004170,0.000060,0.000121,0.376526,0.373021,0.509670,0.490330,0.288952,0.214614,0.271365,0.225070,1
3,200426125,2004,641977.0,1,-1,5.7,48100.0,0.024143,0.024681,0.060394,0.051156,0.013688,0.014545,0.001467,0.001448,0.000150,0.000142,0.416991,0.404963,0.510614,0.489386,0.271320,0.266428,0.305248,0.157004,1
4,200828149,2008,21709.0,0,1,6.3,0.0,0.004149,0.003722,0.249212,0.212253,0.008034,0.010109,0.001322,0.001546,0.000020,0.000102,0.267417,0.253422,0.525334,0.474666,0.295202,0.246751,0.287045,0.171003,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,20085105,2008,4279.0,1,0,5.6,0.0,0.000953,0.000762,0.009623,0.008670,0.008479,0.010766,0.003430,0.003049,0.000095,0.000000,0.481517,0.478659,0.502096,0.497904,0.260290,0.227325,0.290682,0.221704,1
429,20046081,2004,284077.0,0,0,4.8,62147.0,0.118791,0.107160,0.016958,0.017430,0.110334,0.114978,0.003011,0.003278,0.007585,0.007422,0.344353,0.342763,0.506701,0.493299,0.247110,0.289754,0.292977,0.170159,1
430,20006025,2000,28937.0,1,0,17.5,24908.0,0.011536,0.013054,0.010869,0.032332,0.359043,0.363321,0.011297,0.010258,0.000660,0.000773,0.439108,0.460937,0.477869,0.522131,0.345757,0.295593,0.225863,0.132786,1
431,200026159,2000,29482.0,1,0,4.1,0.0,0.001784,0.001430,0.028004,0.025630,0.033579,0.040229,0.004958,0.004722,0.000079,0.000052,0.461352,0.456249,0.503810,0.496190,0.308871,0.246698,0.280644,0.163788,1


In [121]:
modelx = LogisticRegression()

In [122]:
#Recursive Feature Elimination
from sklearn.feature_selection import RFE


rfe = RFE(modelx, 20)
rfe = rfe.fit(X_train, y_train.values.ravel())

values = rfe.support_.tolist()

cols = []

for val, column in zip(values, X_train.columns):
    if val == True:
        cols.append(column)
        
cols

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: Futur

['YEAR_FIPS',
 'YEAR',
 'COUNTY_TOTALVOTES',
 'HOUSE_WINNING_BINARY',
 'SENATE_WINNING_BINARY',
 'UNEMPLOYMENT_RATE',
 'AVG_WAGE_SALARY',
 'BA_FEMALE%',
 'BA_MALE%',
 'H_FEMALE%',
 'H_MALE%',
 'IA_FEMALE%',
 'IA_MALE%',
 'WA_FEMALE%',
 'WA_MALE%',
 'TOT_FEMALE%',
 'TOT_MALE%',
 'TOT_POP_LESS19%',
 'TOT_POP_20to39%',
 'TRAIN']

In [123]:
#split by: train=1; significant columns
X_train, X_test, y_train, y_test = train_test_split_by_year_2(data, year, True, cols)

X_test: (957, 20)
y_test: (957, 1)
X_train: (433, 20)
y_train: (433, 1)


/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:33: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [124]:
#model = LogisticRegression(C=0.0001, solver='lbfgs', dual=False, max_iter=5000)

In [125]:
model4 = LogisticRegression()

In [126]:
model4.fit(X_test, y_test)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [127]:
preditions = model4.predict(X_test)

In [128]:
print("Experiment#4: {:.3f}%".format(accuracy_score(y_test, predictions) * 100))

Experiment#4: 16.510%


In [63]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       158
           1       1.00      0.98      0.99       799

   micro avg       0.99      0.99      0.99       957
   macro avg       0.96      0.99      0.98       957
weighted avg       0.99      0.99      0.99       957



In [64]:
confusion_matrix(y_test, predictions)

array([[158,   0],
       [ 12, 787]])

# Predict 2020

In [151]:
data_2020 = pd.read_csv('../../data/F20_phase1_jun28.csv')

In [152]:
data_2020.columns

Index(['Unnamed: 0', 'YEAR_FIPS', 'YEAR', 'STATE_FIPS', 'COUNTY_FIPS',
       'HOUSE_WINNING_BINARY', 'SENATE_WINNING_BINARY', 'UNEMPLOYMENT_RATE',
       'AVG_WAGE_SALARY', 'AA_FEMALE', 'AA_MALE', 'BA_FEMALE', 'BA_MALE',
       'H_FEMALE', 'H_MALE', 'IA_FEMALE', 'IA_MALE', 'NA_FEMALE', 'NA_MALE',
       'TOT_FEMALE', 'TOT_MALE', 'TOT_POP', 'WA_FEMALE', 'WA_MALE',
       'TOT_POP_LESS19', 'TOT_MALE_LESS19', 'TOT_FEMALE_LESS19',
       'TOT_POP_20to39', 'TOT_MALE_20to39', 'TOT_FEMALE_20to39',
       'TOT_POP_40to59', 'TOT_MALE_40to59', 'TOT_FEMALE_40to59',
       'TOT_POP_Above60', 'TOT_MALE_Above60', 'TOT_FEMALE_Above60',
       'AA_FEMALE%', 'AA_MALE%', 'BA_FEMALE%', 'BA_MALE%', 'H_FEMALE%',
       'H_MALE%', 'IA_FEMALE%', 'IA_MALE%', 'NA_FEMALE%', 'NA_MALE%',
       'WA_FEMALE%', 'WA_MALE%', 'TOT_FEMALE%', 'TOT_MALE%', 'TOT_POP_LESS19%',
       'TOT_POP_20to39%', 'TOT_POP_40to59%', 'TOT_POP_Above60%',
       'COUNTY_TOTALVOTES'],
      dtype='object')

In [153]:
data_2020.drop(['Unnamed: 0', 'YEAR_FIPS'], axis=1, inplace=True)

In [154]:
data_2020.columns

Index(['YEAR', 'STATE_FIPS', 'COUNTY_FIPS', 'HOUSE_WINNING_BINARY',
       'SENATE_WINNING_BINARY', 'UNEMPLOYMENT_RATE', 'AVG_WAGE_SALARY',
       'AA_FEMALE', 'AA_MALE', 'BA_FEMALE', 'BA_MALE', 'H_FEMALE', 'H_MALE',
       'IA_FEMALE', 'IA_MALE', 'NA_FEMALE', 'NA_MALE', 'TOT_FEMALE',
       'TOT_MALE', 'TOT_POP', 'WA_FEMALE', 'WA_MALE', 'TOT_POP_LESS19',
       'TOT_MALE_LESS19', 'TOT_FEMALE_LESS19', 'TOT_POP_20to39',
       'TOT_MALE_20to39', 'TOT_FEMALE_20to39', 'TOT_POP_40to59',
       'TOT_MALE_40to59', 'TOT_FEMALE_40to59', 'TOT_POP_Above60',
       'TOT_MALE_Above60', 'TOT_FEMALE_Above60', 'AA_FEMALE%', 'AA_MALE%',
       'BA_FEMALE%', 'BA_MALE%', 'H_FEMALE%', 'H_MALE%', 'IA_FEMALE%',
       'IA_MALE%', 'NA_FEMALE%', 'NA_MALE%', 'WA_FEMALE%', 'WA_MALE%',
       'TOT_FEMALE%', 'TOT_MALE%', 'TOT_POP_LESS19%', 'TOT_POP_20to39%',
       'TOT_POP_40to59%', 'TOT_POP_Above60%', 'COUNTY_TOTALVOTES'],
      dtype='object')

In [155]:
Index(['YEAR_FIPS', 'YEAR', 'COUNTY_TOTALVOTES', 'HOUSE_WINNING_BINARY',
       'SENATE_WINNING_BINARY', 'UNEMPLOYMENT_RATE', 'AVG_WAGE_SALARY',
       'AA_FEMALE', 'AA_MALE', 'BA_FEMALE', 'BA_MALE', 'H_FEMALE', 'H_MALE',
       'IA_FEMALE', 'IA_MALE', 'NA_FEMALE', 'NA_MALE', 'TOT_FEMALE',
       'TOT_MALE', 'TOT_POP', 'WA_FEMALE', 'WA_MALE', 'TOT_POP_LESS19',
       'TOT_MALE_LESS19', 'TOT_FEMALE_LESS19', 'TOT_POP_20to39',
       'TOT_MALE_20to39', 'TOT_FEMALE_20to39', 'TOT_POP_40to59',
       'TOT_MALE_40to59', 'TOT_FEMALE_40to59', 'TOT_POP_Above60',
       'TOT_MALE_Above60', 'TOT_FEMALE_Above60', 'AA_FEMALE%', 'AA_MALE%',
       'BA_FEMALE%', 'BA_MALE%', 'H_FEMALE%', 'H_MALE%', 'IA_FEMALE%',
       'IA_MALE%', 'NA_FEMALE%', 'NA_MALE%', 'WA_FEMALE%', 'WA_MALE%',
       'TOT_FEMALE%', 'TOT_MALE%', 'TOT_POP_LESS19%', 'TOT_POP_20to39%',
       'TOT_POP_40to59%', 'TOT_POP_Above60%', 'TRAIN'],
      dtype='object')

NameError: name 'Index' is not defined

In [156]:
y_pred = model1.predict(data_2020)

In [157]:
data_2020['WINNING_PARTY_BINARY'] = y_pred

In [160]:
data_2020

,YEAR,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY
0,2020,1,1,1,-1,5.1,36564,380,301,5762,5153,789,860,146,121,29,33,28606,26995,55601,21801,20859,14424,7397,7027,13967,6885,7082,15320,7430,7890,11890,5283,6607,0.006834,0.005414,0.103631,0.092678,0.014190,0.015467,0.002626,0.002176,0.000522,0.000594,0.392097,0.375155,0.514487,0.485513,0.259420,0.251201,0.275535,0.213845,24758.491022,1
1,2020,1,3,1,-1,5.3,69804,1575,933,10092,9400,4829,5302,798,886,73,73,112365,105657,218022,97830,92471,51670,26101,25569,48792,24036,24756,57324,27532,29792,60236,27988,32248,0.007224,0.004279,0.046289,0.043115,0.022149,0.024319,0.003660,0.004064,0.000335,0.000335,0.448716,0.424136,0.515384,0.484616,0.236994,0.223794,0.262928,0.276284,108961.893451,1
2,2020,1,5,1,-1,8.3,70238,61,52,5672,6370,480,584,67,97,16,30,11748,13133,24881,5799,6410,5692,2907,2785,6469,3919,2550,6352,3469,2883,6368,2838,3530,0.002452,0.002090,0.227965,0.256019,0.019292,0.023472,0.002693,0.003899,0.000643,0.001206,0.233069,0.257626,0.472168,0.527832,0.228769,0.259998,0.255295,0.255938,10973.704487,1
3,2020,1,7,1,-1,6.4,79056,34,19,1816,2954,253,335,50,48,4,22,10479,11921,22400,8449,8762,5052,2660,2392,6142,3633,2509,6153,3330,2823,5053,2298,2755,0.001518,0.000848,0.081071,0.131875,0.011295,0.014955,0.002232,0.002143,0.000179,0.000982,0.377188,0.391161,0.467813,0.532188,0.225536,0.274196,0.274687,0.225580,7860.604028,1
4,2020,1,9,1,-1,5.4,66760,103,82,443,507,2597,2939,183,195,26,44,29340,28500,57840,28149,27307,14665,7366,7299,13626,6933,6693,15318,7665,7653,14231,6536,7695,0.001781,0.001418,0.007659,0.008766,0.044900,0.050813,0.003164,0.003371,0.000450,0.000761,0.486670,0.472113,0.507261,0.492739,0.253544,0.235581,0.264834,0.246041,28320.852715,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,2020,56,37,1,1,6.1,57278,253,182,264,328,3321,3603,343,322,34,34,20899,22152,43051,19563,20834,12430,6372,6058,11936,6151,5785,10595,5537,5058,8090,4092,3998,0.005877,0.004228,0.006132,0.007619,0.077141,0.083691,0.007967,0.007480,0.000790,0.000790,0.454415,0.483938,0.485447,0.514553,0.288727,0.277253,0.246103,0.187917,16044.010520,1
3138,2020,56,39,1,1,3.5,0,194,132,75,100,1651,1783,101,110,15,20,11172,11909,23081,10614,11357,4569,2334,2235,7060,3662,3398,6351,3326,3025,5101,2587,2514,0.008405,0.005719,0.003249,0.004333,0.071531,0.077250,0.004376,0.004766,0.000650,0.000867,0.459859,0.492050,0.484034,0.515966,0.197955,0.305879,0.275161,0.221004,9328.764719,1
3139,2020,56,41,1,1,5.8,40941,55,41,90,78,922,953,156,132,13,17,10014,10285,20299,9538,9818,6326,3283,3043,4855,2417,2438,4824,2444,2380,4294,2141,2153,0.002709,0.002020,0.004434,0.003843,0.045421,0.046948,0.007685,0.006503,0.000640,0.000837,0.469875,0.483669,0.493325,0.506675,0.311641,0.239174,0.237647,0.211538,6775.716347,1
3140,2020,56,43,1,1,4.8,41058,37,28,15,25,540,568,78,65,3,5,3897,3988,7885,3672,3778,1971,1017,954,1618,822,796,1985,1012,973,2311,1137,1174,0.004692,0.003551,0.001902,0.003171,0.068484,0.072036,0.009892,0.008244,0.000380,0.000634,0.465694,0.479138,0.494230,0.505770,0.249968,0.205200,0.251744,0.293088,4228.478633,1


In [162]:
#save to csv
data_2020.to_csv("../../data/F20_predictions_jun28.csv")

In [165]:
data_2020.loc[data_2020['HOUSE_WINNING_BINARY'] == 0]

,YEAR,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY
186,2020,6,1,0,0,7.7,46635,275235,253975,97364,88566,184368,188672,8793,9180,8301,7569,846708,820045,1666753,411943,417023,381081,194442,186639,512769,257278,255491,446591,221092,225499,326312,147233,179079,0.165132,0.152377,0.058415,0.053137,0.110615,0.113197,0.005276,0.005508,0.004980,0.004541,0.247153,0.250201,0.507998,0.492002,0.228637,0.307645,0.267941,0.195777,605063.497814,0
187,2020,6,3,0,0,4.3,53577,7,15,1,0,56,69,131,132,0,0,513,588,1101,356,414,219,119,100,205,121,84,294,151,143,383,197,186,0.006358,0.013624,0.000908,0.000000,0.050863,0.062670,0.118983,0.119891,0.000000,0.000000,0.323342,0.376022,0.465940,0.534060,0.198910,0.186194,0.267030,0.347866,7727.247955,1
188,2020,6,5,0,0,4.8,0,369,316,104,957,2097,3564,367,546,50,54,17958,21425,39383,16428,18884,6562,3405,3157,8256,4991,3265,10591,6273,4318,13974,6756,7218,0.009370,0.008024,0.002641,0.024300,0.053246,0.090496,0.009319,0.013864,0.001270,0.001371,0.417134,0.479496,0.455984,0.544016,0.166620,0.209634,0.268923,0.354823,19593.944876,0
189,2020,6,7,0,0,4.0,489208,5797,5638,1810,2368,18941,19587,2917,2811,327,335,116914,114342,231256,100598,98074,54165,27684,26481,68855,35383,33472,50204,24713,25491,58032,26562,31470,0.025067,0.024380,0.007827,0.010240,0.081905,0.084698,0.012614,0.012155,0.001414,0.001449,0.435007,0.424093,0.505561,0.494439,0.234221,0.297744,0.217093,0.250943,93934.598417,1
190,2020,6,9,0,0,4.5,55229,443,348,175,273,2792,2851,465,419,61,54,22905,22697,45602,20860,20734,8511,4428,4083,8680,4487,4193,11350,5522,5828,17061,8260,8801,0.009714,0.007631,0.003838,0.005987,0.061225,0.062519,0.010197,0.009188,0.001338,0.001184,0.457436,0.454673,0.502281,0.497719,0.186637,0.190343,0.248893,0.374128,22385.006775,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114,2020,55,133,0,0,3.4,54792,8134,7532,3250,3623,9541,9754,665,641,113,107,204883,198189,403072,189747,183369,96096,49265,46831,89891,45353,44538,112133,55266,56867,104952,48305,56647,0.020180,0.018686,0.008063,0.008988,0.023671,0.024199,0.001650,0.001590,0.000280,0.000265,0.470752,0.454929,0.508304,0.491696,0.238409,0.223015,0.278196,0.260380,198232.495521,1
3115,2020,55,135,0,0,3.9,38228,168,119,75,200,791,875,191,216,7,4,25188,25940,51128,24449,25079,11372,5914,5458,10885,5587,5298,14141,7193,6948,14730,7246,7484,0.003286,0.002327,0.001467,0.003912,0.015471,0.017114,0.003736,0.004225,0.000137,0.000078,0.478192,0.490514,0.492646,0.507354,0.222422,0.212897,0.276580,0.288100,24673.278451,1
3116,2020,55,137,0,0,5.0,33072,90,48,48,471,775,749,96,98,8,9,11499,12764,24263,11110,11966,4864,2448,2416,4711,2724,1987,6641,3577,3064,8047,4015,4032,0.003709,0.001978,0.001978,0.019412,0.031942,0.030870,0.003957,0.004039,0.000330,0.000371,0.457899,0.493179,0.473932,0.526068,0.200470,0.194164,0.273709,0.331657,16108.908204,1
3117,2020,55,139,0,0,3.6,0,2720,2489,1333,2626,3503,3680,577,699,45,46,84957,86063,171020,78808,78602,40404,20470,19934,48268,25302,22966,43409,22352,21057,38939,17939,21000,0.015905,0.014554,0.007794,0.015355,0.020483,0.021518,0.003374,0.004087,0.000263,0.000269,0.460812,0.459607,0.496766,0.503234,0.236253,0.282236,0.253824,0.227687,84955.183267,0


In [192]:
## now use this to predict the total votes in each county using Linear Regression

In [286]:
votes_historcail_df = pd.read_csv("../../data/combined_jun22_train.csv", encoding = "ISO-8859-1")
votes_df = pd.read_csv("../../data/F20_predictions_jun28.csv")

In [287]:
votes_historcail_df['REP_VOTES'] = votes_historcail_df['REP_VOTES'].astype(np.int64)

In [288]:
votes_historcail_df.drop(['ID', 'Unnamed: 0', 'YEAR_FIPS', 'WINNING_CANDIDATE', 'REP_CANDIDATE', 'DEM_CANDIDATE', 'WINNING_PARTY', 'COUNTY', 'STATE', 'DEM_VOTES', 'TRAIN', 'MARGIN_VICTORY'], axis=1, inplace=True)

In [289]:
X = votes_historcail_df.drop(['REP_VOTES'], axis=1)
y = votes_historcail_df[['REP_VOTES']].values

In [290]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [291]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

In [292]:
linear_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [293]:
y_pred = linear_model.predict(X_test)

In [294]:
print("Accuracy score={:.2f}".format(linear_model.score(X_test, y_test) * 100))

Accuracy score=95.70


In [295]:
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df.head(25)

,Actual,Predicted
0,9384,8744.610030
1,13461,15860.846631
2,7755,8642.062957
3,23698,30500.266122
4,3012,3751.923605
5,3568,4083.212931
6,1590,2632.609595
7,21512,19135.920649
8,6213,6818.825223
9,10266,12765.732146


In [298]:
votes_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [299]:
## now apply 2020 data...

y_2020_pred = linear_model.predict(votes_df)

In [300]:
#merge predictions with the 2020 dataset. 
data_2020['REP_VOTES'] = y_2020_pred

In [304]:
data_2020['COUNTY_TOTALVOTES'] = data_2020['COUNTY_TOTALVOTES'].astype(np.int64)
data_2020['REP_VOTES'] = data_2020['REP_VOTES'].astype(np.int64)

In [305]:
data_2020

,YEAR,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,REP_VOTES
0,2020,1,1,1,-1,5.1,36564,380,301,5762,5153,789,860,146,121,29,33,28606,26995,55601,21801,20859,14424,7397,7027,13967,6885,7082,15320,7430,7890,11890,5283,6607,0.006834,0.005414,0.103631,0.092678,0.014190,0.015467,0.002626,0.002176,0.000522,0.000594,0.392097,0.375155,0.514487,0.485513,0.259420,0.251201,0.275535,0.213845,24758,1,93633063
1,2020,1,3,1,-1,5.3,69804,1575,933,10092,9400,4829,5302,798,886,73,73,112365,105657,218022,97830,92471,51670,26101,25569,48792,24036,24756,57324,27532,29792,60236,27988,32248,0.007224,0.004279,0.046289,0.043115,0.022149,0.024319,0.003660,0.004064,0.000335,0.000335,0.448716,0.424136,0.515384,0.484616,0.236994,0.223794,0.262928,0.276284,108961,1,390332223
2,2020,1,5,1,-1,8.3,70238,61,52,5672,6370,480,584,67,97,16,30,11748,13133,24881,5799,6410,5692,2907,2785,6469,3919,2550,6352,3469,2883,6368,2838,3530,0.002452,0.002090,0.227965,0.256019,0.019292,0.023472,0.002693,0.003899,0.000643,0.001206,0.233069,0.257626,0.472168,0.527832,0.228769,0.259998,0.255295,0.255938,10973,1,55093113
3,2020,1,7,1,-1,6.4,79056,34,19,1816,2954,253,335,50,48,4,22,10479,11921,22400,8449,8762,5052,2660,2392,6142,3633,2509,6153,3330,2823,5053,2298,2755,0.001518,0.000848,0.081071,0.131875,0.011295,0.014955,0.002232,0.002143,0.000179,0.000982,0.377188,0.391161,0.467813,0.532188,0.225536,0.274196,0.274687,0.225580,7860,1,46477513
4,2020,1,9,1,-1,5.4,66760,103,82,443,507,2597,2939,183,195,26,44,29340,28500,57840,28149,27307,14665,7366,7299,13626,6933,6693,15318,7665,7653,14231,6536,7695,0.001781,0.001418,0.007659,0.008766,0.044900,0.050813,0.003164,0.003371,0.000450,0.000761,0.486670,0.472113,0.507261,0.492739,0.253544,0.235581,0.264834,0.246041,28320,1,113371618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,2020,56,37,1,1,6.1,57278,253,182,264,328,3321,3603,343,322,34,34,20899,22152,43051,19563,20834,12430,6372,6058,11936,6151,5785,10595,5537,5058,8090,4092,3998,0.005877,0.004228,0.006132,0.007619,0.077141,0.083691,0.007967,0.007480,0.000790,0.000790,0.454415,0.483938,0.485447,0.514553,0.288727,0.277253,0.246103,0.187917,16044,1,69009100
3138,2020,56,39,1,1,3.5,0,194,132,75,100,1651,1783,101,110,15,20,11172,11909,23081,10614,11357,4569,2334,2235,7060,3662,3398,6351,3326,3025,5101,2587,2514,0.008405,0.005719,0.003249,0.004333,0.071531,0.077250,0.004376,0.004766,0.000650,0.000867,0.459859,0.492050,0.484034,0.515966,0.197955,0.305879,0.275161,0.221004,9328,1,31576572
3139,2020,56,41,1,1,5.8,40941,55,41,90,78,922,953,156,132,13,17,10014,10285,20299,9538,9818,6326,3283,3043,4855,2417,2438,4824,2444,2380,4294,2141,2153,0.002709,0.002020,0.004434,0.003843,0.045421,0.046948,0.007685,0.006503,0.000640,0.000837,0.469875,0.483669,0.493325,0.506675,0.311641,0.239174,0.237647,0.211538,6775,1,33083570
3140,2020,56,43,1,1,4.8,41058,37,28,15,25,540,568,78,65,3,5,3897,3988,7885,3672,3778,1971,1017,954,1618,822,796,1985,1012,973,2311,1137,1174,0.004692,0.003551,0.001902,0.003171,0.068484,0.072036,0.009892,0.008244,0.000380,0.000634,0.465694,0.479138,0.494230,0.505770,0.249968,0.205200,0.251744,0.293088,4228,1,24357278


In [303]:
24758.491022 - 93633063

-93608304.508978

In [ ]:
###-------- take 2 -------##

In [343]:
votes_historcail_df = pd.read_csv("../../data/combined_jun22_train.csv", encoding = "ISO-8859-1")
votes_df = pd.read_csv("../../data/F20_predictions_jun28.csv")

In [344]:
votes_historcail_df_copy = votes_historcail_df[['STATE_FIPS', 'COUNTY_FIPS', 'COUNTY_TOTALVOTES', 'WINNING_PARTY_BINARY', 'REP_VOTES']]

In [345]:
votes_historcail_df_copy

,STATE_FIPS,COUNTY_FIPS,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,REP_VOTES
0,1,1,23932.0,1,17379.0
1,1,3,85338.0,1,66016.0
2,1,5,11509.0,0,5550.0
3,1,7,8420.0,1,6132.0
4,1,9,24006.0,1,20757.0
...,...,...,...,...,...
15564,9,7,87155.0,0,32907.0
15565,9,9,381739.0,0,144465.0
15566,9,11,124765.0,0,48460.0
15567,9,13,75406.0,0,29262.0


In [346]:
votes_historcail_df_copy['COUNTY_TOTALVOTES'] = votes_historcail_df_copy['COUNTY_TOTALVOTES'].astype(np.int64)
votes_historcail_df_copy['REP_VOTES'] = votes_historcail_df_copy['REP_VOTES'].astype(np.int64)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [347]:
votes_historcail_df_copy

,STATE_FIPS,COUNTY_FIPS,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,REP_VOTES
0,1,1,23932,1,17379
1,1,3,85338,1,66016
2,1,5,11509,0,5550
3,1,7,8420,1,6132
4,1,9,24006,1,20757
...,...,...,...,...,...
15564,9,7,87155,0,32907
15565,9,9,381739,0,144465
15566,9,11,124765,0,48460
15567,9,13,75406,0,29262


In [348]:
X = votes_historcail_df_copy.drop(['REP_VOTES'], axis=1)
y = votes_historcail_df_copy[['REP_VOTES']].values

In [349]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [350]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [351]:
y_pred = linear_model.predict(X_test)

In [352]:
print("Accuracy score={:.2f}".format(linear_model.score(X_test, y_test) * 100))

Accuracy score=90.44


In [356]:
votes_df_copy = votes_df[['STATE_FIPS', 'COUNTY_FIPS', 'COUNTY_TOTALVOTES', 'WINNING_PARTY_BINARY']]

In [357]:
y_pred_2020 = linear_model.predict(votes_df_copy)

In [360]:
data_2020['REP_VOTES'] = y_pred_2020
data_2020['REP_VOTES'] = data_2020['REP_VOTES'].astype(np.int64)

In [361]:
data_2020

,YEAR,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,REP_VOTES
0,2020,1,1,1,-1,5.1,36564,380,301,5762,5153,789,860,146,121,29,33,28606,26995,55601,21801,20859,14424,7397,7027,13967,6885,7082,15320,7430,7890,11890,5283,6607,0.006834,0.005414,0.103631,0.092678,0.014190,0.015467,0.002626,0.002176,0.000522,0.000594,0.392097,0.375155,0.514487,0.485513,0.259420,0.251201,0.275535,0.213845,24758,1,14345
1,2020,1,3,1,-1,5.3,69804,1575,933,10092,9400,4829,5302,798,886,73,73,112365,105657,218022,97830,92471,51670,26101,25569,48792,24036,24756,57324,27532,29792,60236,27988,32248,0.007224,0.004279,0.046289,0.043115,0.022149,0.024319,0.003660,0.004064,0.000335,0.000335,0.448716,0.424136,0.515384,0.484616,0.236994,0.223794,0.262928,0.276284,108961,1,44772
2,2020,1,5,1,-1,8.3,70238,61,52,5672,6370,480,584,67,97,16,30,11748,13133,24881,5799,6410,5692,2907,2785,6469,3919,2550,6352,3469,2883,6368,2838,3530,0.002452,0.002090,0.227965,0.256019,0.019292,0.023472,0.002693,0.003899,0.000643,0.001206,0.233069,0.257626,0.472168,0.527832,0.228769,0.259998,0.255295,0.255938,10973,1,9369
3,2020,1,7,1,-1,6.4,79056,34,19,1816,2954,253,335,50,48,4,22,10479,11921,22400,8449,8762,5052,2660,2392,6142,3633,2509,6153,3330,2823,5053,2298,2755,0.001518,0.000848,0.081071,0.131875,0.011295,0.014955,0.002232,0.002143,0.000179,0.000982,0.377188,0.391161,0.467813,0.532188,0.225536,0.274196,0.274687,0.225580,7860,1,8247
4,2020,1,9,1,-1,5.4,66760,103,82,443,507,2597,2939,183,195,26,44,29340,28500,57840,28149,27307,14665,7366,7299,13626,6933,6693,15318,7665,7653,14231,6536,7695,0.001781,0.001418,0.007659,0.008766,0.044900,0.050813,0.003164,0.003371,0.000450,0.000761,0.486670,0.472113,0.507261,0.492739,0.253544,0.235581,0.264834,0.246041,28320,1,15641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,2020,56,37,1,1,6.1,57278,253,182,264,328,3321,3603,343,322,34,34,20899,22152,43051,19563,20834,12430,6372,6058,11936,6151,5785,10595,5537,5058,8090,4092,3998,0.005877,0.004228,0.006132,0.007619,0.077141,0.083691,0.007967,0.007480,0.000790,0.000790,0.454415,0.483938,0.485447,0.514553,0.288727,0.277253,0.246103,0.187917,16044,1,11717
3138,2020,56,39,1,1,3.5,0,194,132,75,100,1651,1783,101,110,15,20,11172,11909,23081,10614,11357,4569,2334,2235,7060,3662,3398,6351,3326,3025,5101,2587,2514,0.008405,0.005719,0.003249,0.004333,0.071531,0.077250,0.004376,0.004766,0.000650,0.000867,0.459859,0.492050,0.484034,0.515966,0.197955,0.305879,0.275161,0.221004,9328,1,9293
3139,2020,56,41,1,1,5.8,40941,55,41,90,78,922,953,156,132,13,17,10014,10285,20299,9538,9818,6326,3283,3043,4855,2417,2438,4824,2444,2380,4294,2141,2153,0.002709,0.002020,0.004434,0.003843,0.045421,0.046948,0.007685,0.006503,0.000640,0.000837,0.469875,0.483669,0.493325,0.506675,0.311641,0.239174,0.237647,0.211538,6775,1,8373
3140,2020,56,43,1,1,4.8,41058,37,28,15,25,540,568,78,65,3,5,3897,3988,7885,3672,3778,1971,1017,954,1618,822,796,1985,1012,973,2311,1137,1174,0.004692,0.003551,0.001902,0.003171,0.068484,0.072036,0.009892,0.008244,0.000380,0.000634,0.465694,0.479138,0.494230,0.505770,0.249968,0.205200,0.251744,0.293088,4228,1,7454


In [362]:
#info D total votes. 
data_2020['DEM_VOTES'] = data_2020['COUNTY_TOTALVOTES'] - data_2020['REP_VOTES']

In [363]:
data_2020

,YEAR,STATE_FIPS,COUNTY_FIPS,HOUSE_WINNING_BINARY,SENATE_WINNING_BINARY,UNEMPLOYMENT_RATE,AVG_WAGE_SALARY,AA_FEMALE,AA_MALE,BA_FEMALE,BA_MALE,H_FEMALE,H_MALE,IA_FEMALE,IA_MALE,NA_FEMALE,NA_MALE,TOT_FEMALE,TOT_MALE,TOT_POP,WA_FEMALE,WA_MALE,TOT_POP_LESS19,TOT_MALE_LESS19,TOT_FEMALE_LESS19,TOT_POP_20to39,TOT_MALE_20to39,TOT_FEMALE_20to39,TOT_POP_40to59,TOT_MALE_40to59,TOT_FEMALE_40to59,TOT_POP_Above60,TOT_MALE_Above60,TOT_FEMALE_Above60,AA_FEMALE%,AA_MALE%,BA_FEMALE%,BA_MALE%,H_FEMALE%,H_MALE%,IA_FEMALE%,IA_MALE%,NA_FEMALE%,NA_MALE%,WA_FEMALE%,WA_MALE%,TOT_FEMALE%,TOT_MALE%,TOT_POP_LESS19%,TOT_POP_20to39%,TOT_POP_40to59%,TOT_POP_Above60%,COUNTY_TOTALVOTES,WINNING_PARTY_BINARY,REP_VOTES,DEM_VOTES
0,2020,1,1,1,-1,5.1,36564,380,301,5762,5153,789,860,146,121,29,33,28606,26995,55601,21801,20859,14424,7397,7027,13967,6885,7082,15320,7430,7890,11890,5283,6607,0.006834,0.005414,0.103631,0.092678,0.014190,0.015467,0.002626,0.002176,0.000522,0.000594,0.392097,0.375155,0.514487,0.485513,0.259420,0.251201,0.275535,0.213845,24758,1,14345,10413
1,2020,1,3,1,-1,5.3,69804,1575,933,10092,9400,4829,5302,798,886,73,73,112365,105657,218022,97830,92471,51670,26101,25569,48792,24036,24756,57324,27532,29792,60236,27988,32248,0.007224,0.004279,0.046289,0.043115,0.022149,0.024319,0.003660,0.004064,0.000335,0.000335,0.448716,0.424136,0.515384,0.484616,0.236994,0.223794,0.262928,0.276284,108961,1,44772,64189
2,2020,1,5,1,-1,8.3,70238,61,52,5672,6370,480,584,67,97,16,30,11748,13133,24881,5799,6410,5692,2907,2785,6469,3919,2550,6352,3469,2883,6368,2838,3530,0.002452,0.002090,0.227965,0.256019,0.019292,0.023472,0.002693,0.003899,0.000643,0.001206,0.233069,0.257626,0.472168,0.527832,0.228769,0.259998,0.255295,0.255938,10973,1,9369,1604
3,2020,1,7,1,-1,6.4,79056,34,19,1816,2954,253,335,50,48,4,22,10479,11921,22400,8449,8762,5052,2660,2392,6142,3633,2509,6153,3330,2823,5053,2298,2755,0.001518,0.000848,0.081071,0.131875,0.011295,0.014955,0.002232,0.002143,0.000179,0.000982,0.377188,0.391161,0.467813,0.532188,0.225536,0.274196,0.274687,0.225580,7860,1,8247,-387
4,2020,1,9,1,-1,5.4,66760,103,82,443,507,2597,2939,183,195,26,44,29340,28500,57840,28149,27307,14665,7366,7299,13626,6933,6693,15318,7665,7653,14231,6536,7695,0.001781,0.001418,0.007659,0.008766,0.044900,0.050813,0.003164,0.003371,0.000450,0.000761,0.486670,0.472113,0.507261,0.492739,0.253544,0.235581,0.264834,0.246041,28320,1,15641,12679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,2020,56,37,1,1,6.1,57278,253,182,264,328,3321,3603,343,322,34,34,20899,22152,43051,19563,20834,12430,6372,6058,11936,6151,5785,10595,5537,5058,8090,4092,3998,0.005877,0.004228,0.006132,0.007619,0.077141,0.083691,0.007967,0.007480,0.000790,0.000790,0.454415,0.483938,0.485447,0.514553,0.288727,0.277253,0.246103,0.187917,16044,1,11717,4327
3138,2020,56,39,1,1,3.5,0,194,132,75,100,1651,1783,101,110,15,20,11172,11909,23081,10614,11357,4569,2334,2235,7060,3662,3398,6351,3326,3025,5101,2587,2514,0.008405,0.005719,0.003249,0.004333,0.071531,0.077250,0.004376,0.004766,0.000650,0.000867,0.459859,0.492050,0.484034,0.515966,0.197955,0.305879,0.275161,0.221004,9328,1,9293,35
3139,2020,56,41,1,1,5.8,40941,55,41,90,78,922,953,156,132,13,17,10014,10285,20299,9538,9818,6326,3283,3043,4855,2417,2438,4824,2444,2380,4294,2141,2153,0.002709,0.002020,0.004434,0.003843,0.045421,0.046948,0.007685,0.006503,0.000640,0.000837,0.469875,0.483669,0.493325,0.506675,0.311641,0.239174,0.237647,0.211538,6775,1,8373,-1598
3140,2020,56,43,1,1,4.8,41058,37,28,15,25,540,568,78,65,3,5,3897,3988,7885,3672,3778,1971,1017,954,1618,822,796,1985,1012,973,2311,1137,1174,0.004692,0.003551,0.001902,0.003171,0.068484,0.072036,0.009892,0.008244,0.000380,0.000634,0.465694,0.479138,0.494230,0.505770,0.249968,0.205200,0.251744,0.293088,4228,1,7454,-3226


In [364]:
#save to csv
data_2020.to_csv("../../data/F20_predictions_0703_new.csv")